# NLP frequency analysis example speech processing


In [ ]:
import pandas as pd
import warnings
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
from datetime import datetime
warnings.filterwarnings('ignore')
import time
import random

# Download the data from the white house 
Importing all the links from https://www.whitehouse.gov/briefing-room/press-briefings/ 

In [ ]:
base_url = "https://www.whitehouse.gov/briefing-room/press-briefings/page/"
page_number = 1
all_links = []
#Biden administration
while True:
    try:
        url = base_url + str(page_number)
        html = urlopen(url).read()
        soup = BeautifulSoup(html, features="html.parser")
        urls = soup.find_all('a', class_="news-item__title")
        for url in urls:
            all_links.append(url['href'])
        page_number += 1
    except HTTPError:
        break

#Trump administration
base_url = "https://trumpwhitehouse.archives.gov/briefings-statements/page/"
page_number = 1

while True:
    try:
        url = base_url + str(page_number)
        html = urlopen(url).read()
        soup = BeautifulSoup(html, features="html.parser")
        urls = soup.find_all('h2', class_="briefing-statement__title")
        for url in urls:
            all_links.append(url.a['href'])
        page_number += 1
    except HTTPError:
        break

with open('/Resources/url.txt', 'w') as f:
    for link in all_links:
        f.write(link + "\n")

## Importing the speech
Starting with the extraction function which takes as input the link and return the text data. The function work with meta and html tag, that can change thus it is good to check the site to not incut in errors.

In [ ]:
def extr(url, adm):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()  # rip it out

    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = "\n".join(chunk for chunk in chunks if chunk)

    if adm == 1:  # biden administration
        # get metas
        # extract title
        title = soup.find("meta", property="og:title")
        title = title["content"]
        print("Title:" + title)

        # extract description
        desc = soup.find("meta", property="og:description")
        desc = desc["content"]
        print("Description:" + desc)

        # extract datatime
        date = soup.find("meta", property="article:published_time")
        date_str = str(date["content"])

        # convert the date in the correct format
        date = datetime.fromisoformat(date_str)
        print("Date:" + str(date))

        # extract text
        body = soup.find("section", class_="body-content")
        body = body.text
        print("Text extracted")
        time.sleep(random.uniform(0.5,2))
    elif adm == 0:
        # extract title
        title = soup.find("h1", {"class": "page-header__title"}).text
        print("Title:" + title)

        # extract date
        date_str = soup.find("time").text

        # convert the date in the correct format
        date_obj = datetime.strptime(date_str, "%B %d, %Y")
        date = date_obj.isoformat() + "+00:00"
        print("Date:" + str(date))

        # no desc with the trump

        # extract text
        body = soup.find("div", class_="page-content__content editor")
        body = body.text
        print("Text extracted")

        # extract description using library gensim
        desc = "Nan"

    print("----------------------------------------------------")
    print("----------------------------------------------------")
    out = [date, title, desc, body, url]
    return out




1.   Reading the text file with the links
2.   Recall the extrapolation function for each link
3.   Saving the data in a csv file



In [ ]:

# reading the file with the link
f = open("/Resources/url.txt", "r")
url1 = f.readlines()


# print(speech_text)
# create a data frame
# initialize variables
date = []
title = []
desc = []
speech_text = []
link = []
admin = []

# download the data
for url_count in url1:
    print("Analizing:" + url_count)
    if "https://www.whitehouse.gov/" in url_count:  # biden adm
        out = extr(str(url_count), 1)  # recalling function extrapolation
        admin.append("Biden")
    elif "https://trumpwhitehouse.archives.gov/" in url_count:
        out = extr(str(url_count), 0)  # recalling function extrapolation
        admin.append("Trump")

    # append the results
    date.append(out[0])
    title.append(out[1])
    desc.append(out[2])
    speech_text.append(out[3])
    link.append(out[4])

columns_name = [
    "Administration",
    "Date Time",
    "Title",
    "Description",
    "Main text",
    "URL",
]

df_imported = pd.DataFrame(
    list(zip(admin, date, title, desc, speech_text, link)), columns=columns_name
)


# save in a csv file
df_imported.to_csv("1_onlyrawdatadownloaded.csv", index=False)

Output streaming troncato alle ultime 5000 righe.
----------------------------------------------------
Analizing:https://www.whitehouse.gov/briefing-room/press-briefings/2022/10/20/press-gaggle-by-press-secretary-karine-jean-pierre-and-infrastructure-act-implementation-coordinator-mitch-landrieu-en-route-pittsburgh-pennsylvania/

Title:Press Gaggle by Press Secretary Karine Jean-Pierre and Infrastructure Act Implementation Coordinator Mitch Landrieu En Route Pittsburgh, Pennsylvania | The White House
Description:Aboard Air Force OneEn Route Pittsburgh, Pennsylvania 11:57 A.M. EDT MS. JEAN-PIERRE:  Okay.  All right.  As you all know, we’re on our way to Pittsburgh, where the President will visit and deliver remarks at the Fern Hollow Bridge in Pittsburgh’s East End to discuss how — how Bipartisan Infrastructure Law investments are helping to rebuild…
Date:2022-10-20 19:18:48+00:00
Text extracted
----------------------------------------------------
---------------------------------------

Thus I save a csv fil called Book1.csv. Now I open and import the data and starting the text analysis 
! !RUN UP TO HERE IF YOU WANT TO DOWNLOAD THE FULL DATASET !!

In [ ]:
#reading in the data

speech_df = pd.read_csv('1_onlyrawdatadownloaded.csv', sep =",")
speech_df

,Administration,Date Time,Title,Description,Main text,URL
0,Trump,2021-01-20T00:00:00+00:00,Statement from the Press Secretary Regarding E...,Nan,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...,https://trumpwhitehouse.archives.gov/briefings...
1,Trump,2021-01-19T00:00:00+00:00,Statement from National Security Advisor Rober...,Nan,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...,https://trumpwhitehouse.archives.gov/briefings...
2,Trump,2021-01-19T00:00:00+00:00,Text of a Letter to the Speaker of the House o...,Nan,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...,https://trumpwhitehouse.archives.gov/briefings...
3,Trump,2021-01-19T00:00:00+00:00,President Trump Awards Presidential Commendati...,Nan,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...,https://trumpwhitehouse.archives.gov/briefings...
4,Trump,2021-01-19T00:00:00+00:00,Remarks by President Trump In Farewell Address...,Nan,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...,https://trumpwhitehouse.archives.gov/briefings...
...,...,...,...,...,...,...
7427,Biden,2021-01-26 23:09:50+00:00,Press Briefing by Press Secretary Jen Psaki an...,James S. Brady Press Briefing Room 12:39 P.M. ...,\n\n\nJames S. Brady Press Briefing Room\n12:3...,https://www.whitehouse.gov/briefing-room/press...
7428,Biden,2021-01-25 22:59:20+00:00,"Press Briefing by Press Secretary Jen Psaki, J...",James S. Brady Press Briefing Room 1:17 P.M. ...,\n\n\nJames S. Brady Press Briefing Room\n 1:1...,https://www.whitehouse.gov/briefing-room/press...
7429,Biden,2021-01-22 19:06:00+00:00,Press Briefing by Press Secretary Jen Psaki an...,James S. Brady Press Briefing Room 1:03 P.M. E...,\n\n\nJames S. Brady Press Briefing Room\n1:03...,https://www.whitehouse.gov/briefing-room/press...
7430,Biden,2021-01-22 00:35:00+00:00,"Press Briefing by Press Secretary Jen Psaki, J...",James S. Brady Press Briefing Room 4:03 P.M. E...,\n\n\nJames S. Brady Press Briefing Room\n4:03...,https://www.whitehouse.gov/briefing-room/press...


This is the dataframe that has been imported from the csv file

## Removing Duplicates

In [ ]:
#let's remove duplicates first

speech_df['dup'] = speech_df.duplicated(subset=None, keep='first')

In [ ]:
speech_df.head()

,Administration,Date Time,Title,Description,Main text,URL,dup
0,Trump,2021-01-20T00:00:00+00:00,Statement from the Press Secretary Regarding E...,Nan,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...,https://trumpwhitehouse.archives.gov/briefings...,False
1,Trump,2021-01-19T00:00:00+00:00,Statement from National Security Advisor Rober...,Nan,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...,https://trumpwhitehouse.archives.gov/briefings...,False
2,Trump,2021-01-19T00:00:00+00:00,Text of a Letter to the Speaker of the House o...,Nan,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...,https://trumpwhitehouse.archives.gov/briefings...,False
3,Trump,2021-01-19T00:00:00+00:00,President Trump Awards Presidential Commendati...,Nan,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...,https://trumpwhitehouse.archives.gov/briefings...,False
4,Trump,2021-01-19T00:00:00+00:00,Remarks by President Trump In Farewell Address...,Nan,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...,https://trumpwhitehouse.archives.gov/briefings...,False


We have created a new column that stores a boolean value whether the row is a duplicate row or not. We can see that for the second Sherlock Holmes the value in that column is True. We want to delete those rows where dup == True

In [ ]:
speech_df = speech_df[speech_df['dup'] == False]

In [ ]:
speech_df

,Main text
0,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...
1,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...
2,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...
3,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...
4,\n\n\nShare:\n\n\n\n\nShare on Facebook \n\n\n...
...,...
7427,\n\n\nJames S. Brady Press Briefing Room\n12:3...
7428,\n\n\nJames S. Brady Press Briefing Room\n 1:1...
7429,\n\n\nJames S. Brady Press Briefing Room\n1:03...
7430,\n\n\nJames S. Brady Press Briefing Room\n4:03...


In [ ]:
del speech_df['dup'] # deleting "dup" column since we don't need it anymore
speech_df = pd.read_csv('/Full.csv', sep =",") #SAVING THE DATASET